In [18]:
import json
import requests
import pandas as pd

base_url = 'http://127.0.0.1:5000/hcc_risk_models/api/v1.0/models'

In [2]:
url = base_url
print('url: ', url)
result_models = requests.get(url)
print('result_models: ', json.dumps(result_models.json(), indent=2))
print()

url:  http://127.0.0.1:5000/hcc_risk_models/api/v1.0/models
result_models:  {
  "V2216_79_O2": "CMS-HCC 2017 Initial Model, 79 HCC Variables",
  "V2217_79_O1": "CMS-HCC 2017 Midyear Final Model, 79 HCC Variables"
}



In [7]:
model = 'V2217_79_O1'
url = '{}/{}'.format(base_url, model)
print('url: ', url)
result_model = requests.get(url)
print('result_model (keys): ', result_model.json().keys())


url:  http://127.0.0.1:5000/hcc_risk_models/api/v1.0/models/V2217_79_O1
result_model (keys):  dict_keys(['hcc_descriptions', 'icd_to_hcc_mappings', 'model_coefficients', 'model_description', 'model_name', 'model_segments'])


In [59]:
patients = [
    {
        "pt_id": 1001,
        "sex": 1,
        "dob": "1930-8-21",
        "ltimcaid": 1,
        "nemcaid": 0,
        "orec": 2,
        "diagnoses": [
            {
                "diag_code": "A420",
                "diag_type": 0
            },
            {
                "diag_code": "A4150",
                "diag_type": 0
            }
        ]
    },
    {
        "pt_id": 1002,
        "sex": 2,
        "dob": "1927-7-12",
        "ltimcaid": 0,
        "nemcaid": 0,
        "orec": 1,
        "diagnoses": [
            {
                "diag_code": "G030",
                "diag_type": 0
            },
            {
                "diag_code": "C7410",
                "diag_type": 0
            }
        ]
    }
]
url = '{}/{}/evaluate'.format(base_url, model)
print('url: ', url)


url:  http://127.0.0.1:5000/hcc_risk_models/api/v1.0/models/V2217_79_O1/evaluate


In [60]:
response = requests.post(url, json=patients)
json_data = json.loads(response.text)['patients']

In [61]:
hcc_results = pd.DataFrame.from_dict(json_data)
hcc_results

,demographic_data,diagnoses_to_hccs,pt_id,risk_profiles
0,"{'age': 88, 'dob': '1930-08-21', 'ltimcaid': 1...","[{'assign_type': 'primary', 'cc': 115, 'cc_des...",1001,{'CFA': {'demographic_coefficients': {'M85_89'...
1,"{'age': 91, 'dob': '1927-07-12', 'ltimcaid': 0...","[{'assign_type': 'primary', 'cc': 10, 'cc_desc...",1002,{'CFA': {'demographic_coefficients': {'F90_94'...


In [62]:
hcc_results.loc[0]['diagnoses_to_hccs']

[{'assign_type': 'primary',
  'cc': 115,
  'cc_description': 'Pneumococcal Pneumonia, Empyema, Lung Abscess',
  'diag_code': 'A420',
  'diag_description': 'Pulmonary actinomycosis',
  'diag_type': 0,
  'hcc': 115},
 {'assign_type': 'primary',
  'cc': 2,
  'cc_description': 'Septicemia, Sepsis, Systemic Inflammatory Response Syndrome/Shock',
  'diag_code': 'A4150',
  'diag_description': 'Gram-negative sepsis, unspecified',
  'diag_type': 0,
  'hcc': 2}]